In [1]:
import energyflow
import numpy as np

In [2]:
numdata = 200000
numvalid = 10000
numtrain = numdata - numvalid
X, Y = energyflow.datasets.qg_jets.load(num_data=numdata, generator='pythia', pad=True, with_bc=False, cache_dir='~/.energyflow')

In [3]:
# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

In [4]:
# preprocess PIDs so they are O(1) or less
X[:,:,3] = X[:,:,3] / 2000

In [5]:
# Only keep 80 hardest particles
import random
nparticles = 80
X_clipped = np.array([np.flip(x[x[:,0].argsort()][-nparticles:],axis=0) for x in X])
for x_clipped in X_clipped:
    np.random.shuffle(x_clipped)

In [6]:
X_train = X_clipped[:numtrain]
X_valid = X_clipped[numtrain:]
Y_train = Y[:numtrain]
Y_valid = Y[numtrain:]

In [7]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Regular NN

In [8]:
inputs = keras.Input(shape=(nparticles,4))
layer = keras.layers.Conv1D(16,1,padding="same",activation="relu")(inputs)
layer = keras.layers.GlobalAveragePooling1D()(layer)
layer = keras.layers.Lambda(lambda x: x * np.sqrt(nparticles))(layer)    # Make variance after sum the same as before sum
layer = keras.layers.Dense(16,activation="relu")(layer)
output = keras.layers.Dense(1,activation="sigmoid")(layer)
regular_model = keras.Model(inputs=inputs, outputs=output)
regular_model.summary()

regular_model.compile(optimizer=keras.optimizers.Adam(),
             loss = 'binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80, 4)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 80, 16)            80        
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
lambda (Lambda)              (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 369
Trainable params: 369
Non-trainable params: 0
______

In [9]:
history = regular_model.fit(X_train,Y_train,epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
190000/190000 [==============================] - 21s 109us/sample - loss: 0.5019 - acc: 0.7595
Epoch 2/10
190000/190000 [==============================] - 21s 112us/sample - loss: 0.4694 - acc: 0.7825
Epoch 3/10
190000/190000 [==============================] - 24s 128us/sample - loss: 0.4645 - acc: 0.7860
Epoch 4/10
190000/190000 [==============================] - 28s 147us/sample - loss: 0.4610 - acc: 0.7889
Epoch 5/10
190000/190000 [==============================] - 21s 112us/sample - loss: 0.4594 - acc: 0.7902
Epoch 6/10
190000/190000 [==============================] - 23s 119us/sample - loss: 0.4585 - acc: 0.7897
Epoch 7/10
190000/190000 [==============================] - 29s 152us/sample - loss: 0.4571 - acc: 0.7901
Epoch 8/10
190000/190000 [==============================] - 25s 129us/sample - loss: 0.4562 - acc: 0.7913
Epoch 9/10
190000/190000 [==============================] - 22s 117us/sample - loss: 0.4555 - acc: 0.791

## BNN

### 1. Same architecture as regular

#### Build

In [10]:
batch_size = 32
labels = tf.placeholder(tf.float32,shape=[None,1])
learning_rate = 0.001


inputs = keras.Input(shape=(nparticles,4))
layer = tfp.layers.Convolution1DFlipout(16,1,padding="same",activation="relu",name="1_Conv_1")(inputs)
layer = keras.layers.GlobalAveragePooling1D(name="2_Sum")(layer)
layer = keras.layers.Lambda(lambda x: x * np.sqrt(nparticles))(layer)
layer = tfp.layers.DenseFlipout(16,activation="relu",name="3_Dense_1")(layer)
output = tfp.layers.DenseFlipout(1,name="4_Dense_logit")(layer)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

inputs = tf.placeholder(tf.float32,shape=[None,nparticles,4],name="inputs")
logits = model(inputs)

labels_distribution = tfp.distributions.Bernoulli(logits=logits)

neg_log_likelihood = -tf.reduce_mean(labels_distribution.log_prob(labels),axis=-1)
kl = sum(model.losses) / len(X_train)
elbo_loss = neg_log_likelihood + kl

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(elbo_loss)

predictions = tf.round(tf.sigmoid(logits))
accuracy, accuracy_update_op = tf.metrics.accuracy(labels=labels, predictions=predictions)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80, 4)             0         
_________________________________________________________________
1_Conv_1 (Conv1DFlipout)     (None, 80, 16)            144       
_________________________________________________________________
2_Sum (GlobalAveragePooling1 (None, 16)                0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 16)                0         
_________________________________________________________________
3_Dense_1 (DenseFlipout)     (None, 16)                528       
_________________________________________________________________
4_Dense_logit (DenseFlipout) (None, 1)                 33        
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________
Instructions

#### Train

In [11]:
epochs = 10 #number of training steps to run
max_step = int(len(X_train)/batch_size)

init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session() as sess:
        sess.run(init_op)
# Run the training loop.
        for epoch in range(epochs):
            loss_list = []
            acc_list = []
            for step in range(max_step):
                imin = step*batch_size
                x_train = X_train[imin:imin+batch_size]
                y_train = Y_train[imin:imin+batch_size].reshape(batch_size,1)

                _ = sess.run([train_op, accuracy_update_op],
                             feed_dict={inputs: x_train,
                                        labels: y_train})
            
                loss_value, accuracy_value = sess.run([elbo_loss, accuracy],
                                                      feed_dict={inputs: x_train,
                                                                 labels: y_train})
                loss_list.append(loss_value)
                acc_list.append(accuracy_value)

            print("Epoch: {:>3d} Loss: {:.6f} Accuracy: {:.3f}".format(epoch,
                                                                       np.mean(loss_list),
                                                                       np.mean(acc_list)))

Epoch:   0 Loss: 0.541010 Accuracy: 0.704
Epoch:   1 Loss: 0.494689 Accuracy: 0.743
Epoch:   2 Loss: 0.479570 Accuracy: 0.756
Epoch:   3 Loss: 0.473701 Accuracy: 0.763
Epoch:   4 Loss: 0.470408 Accuracy: 0.768
Epoch:   5 Loss: 0.468528 Accuracy: 0.772
Epoch:   6 Loss: 0.466568 Accuracy: 0.774
Epoch:   7 Loss: 0.465176 Accuracy: 0.776
Epoch:   8 Loss: 0.463950 Accuracy: 0.778
Epoch:   9 Loss: 0.462762 Accuracy: 0.779


### 2. Bigger architecture

#### Build

In [12]:
batch_size = 32
labels = tf.placeholder(tf.float32,shape=[None,1])
learning_rate = 0.001


inputs = keras.Input(shape=(nparticles,4))
layer = tfp.layers.Convolution1DFlipout(64,1,padding="same",activation="relu",name="1_Conv_1")(inputs)
layer = tfp.layers.Convolution1DFlipout(64,1,padding="same",activation="relu",name="2_Conv_2")(layer)
layer = keras.layers.GlobalAveragePooling1D(name="3_Sum")(layer)
layer = keras.layers.Lambda(lambda x: x * np.sqrt(nparticles))(layer)
layer = tfp.layers.DenseFlipout(64,activation="relu",name="4_Dense_1")(layer)
layer = tfp.layers.DenseFlipout(64,activation="relu",name="5_Dense_2")(layer)
output = tfp.layers.DenseFlipout(1,name="6_Dense_logit")(layer)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

inputs = tf.placeholder(tf.float32,shape=[None,nparticles,4],name="inputs")
logits = model(inputs)

labels_distribution = tfp.distributions.Bernoulli(logits=logits)

neg_log_likelihood = -tf.reduce_mean(labels_distribution.log_prob(labels),axis=-1)
kl = sum(model.losses) / len(X_train)
elbo_loss = neg_log_likelihood + kl

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(elbo_loss)

predictions = tf.round(tf.sigmoid(logits))
accuracy, accuracy_update_op = tf.metrics.accuracy(labels=labels, predictions=predictions)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 80, 4)             0         
_________________________________________________________________
1_Conv_1 (Conv1DFlipout)     (None, 80, 64)            576       
_________________________________________________________________
2_Conv_2 (Conv1DFlipout)     (None, 80, 64)            8256      
_________________________________________________________________
3_Sum (GlobalAveragePooling1 (None, 64)                0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 64)                0         
_________________________________________________________________
4_Dense_1 (DenseFlipout)     (None, 64)                8256      
_________________________________________________________________
5_Dense_2 (DenseFlipout)     (None, 64)                8256      
__________

#### Train

In [13]:
epochs = 10 #number of training steps to run
max_step = int(len(X_train)/batch_size)

init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session() as sess:
        sess.run(init_op)
# Run the training loop.
        for epoch in range(epochs):
            loss_list = []
            acc_list = []
            for step in range(max_step):
                imin = step*batch_size
                x_train = X_train[imin:imin+batch_size]
                y_train = Y_train[imin:imin+batch_size].reshape(batch_size,1)

                _ = sess.run([train_op, accuracy_update_op],
                             feed_dict={inputs: x_train,
                                        labels: y_train})
            
                loss_value, accuracy_value = sess.run([elbo_loss, accuracy],
                                                      feed_dict={inputs: x_train,
                                                                 labels: y_train})
                loss_list.append(loss_value)
                acc_list.append(accuracy_value)

            print("Epoch: {:>3d} Loss: {:.6f} Accuracy: {:.3f}".format(epoch,
                                                                       np.mean(loss_list),
                                                                       np.mean(acc_list)))

Epoch:   0 Loss: 0.626456 Accuracy: 0.718
Epoch:   1 Loss: 0.561848 Accuracy: 0.765
Epoch:   2 Loss: 0.535312 Accuracy: 0.772
Epoch:   3 Loss: 0.517190 Accuracy: 0.776
Epoch:   4 Loss: 0.505375 Accuracy: 0.779
Epoch:   5 Loss: 0.495442 Accuracy: 0.781
Epoch:   6 Loss: 0.488534 Accuracy: 0.783
Epoch:   7 Loss: 0.482497 Accuracy: 0.784
Epoch:   8 Loss: 0.478142 Accuracy: 0.785
Epoch:   9 Loss: 0.473924 Accuracy: 0.786
